# **Writeup | Behavioral Cloning**   
5 min read

**Abstract —** this notebook is the writeup of the Behavioral Cloning project as part of the SELF-DRIVING CAR nanodegree program. We apply Deep Learning to clone the behavior of a human driver by getting training data from examples of human driving in a simulator. Then these data feed into a convolutional neural network (CNN) to map pixels from camera images directly to steering commands. This way the CNN learns to predict the appropriate steering angle when the car drives in autonomous mode in the simulator.   

The goals of this project are broken down into the following steps:   
```
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report
```

**Note:** the architecture has been inspired by the [nvidia neural network](https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf) before being tweaked a bit using the [Keras](https://keras.io/) deep learning library with Tensorflow backend. The convolutional neural network has been trained only with the sample data provided by Udacity.

[//]: # (Image References)
[image1]: ./examples/placeholder.png "Model Visualization"
[image2]: ./examples/placeholder.png "Grayscaling"
[image3]: ./examples/placeholder_small.png "Recovery Image"
[image4]: ./examples/placeholder_small.png "Recovery Image"
[image5]: ./examples/placeholder_small.png "Recovery Image"
[image6]: ./examples/placeholder_small.png "Normal Image"
[image7]: ./examples/placeholder_small.png "Flipped Image"   

---
## Rubric Points   
*Here I will consider the [rubric points](https://review.udacity.com/#!/rubrics/432/view) individually and describe how I addressed each point in my implementation.*   

### Files Submitted & Code Quality

#### 1. Submission includes all required files and can be used to run the simulator in autonomous mode

My project includes the following files:
```
* model.py containing the script to create and train the model
* tools.py containing the generator to pull pieces of the data and process them on the fly 
* drive.py for driving the car in autonomous mode
* model.h5 containing a trained convolution neural network 
* writeup_report.md or writeup_report.pdf summarizing the results
```   

#### 2. Submission includes functional code
Using the Udacity provided simulator and a modified version of the drive.py file, the car can be driven autonomously around the track by executing 
```sh
python drive.py model.h5
```


#### 3. Submission code is usable and readable

The model.py and tools.py files contains the code for training and saving the convolution neural network. The files show the pipeline I used for training and validating the model, and they contain comments to explain how the code works. 

### Model Architecture and Training Strategy

#### 1. An appropriate model architecture has been employed

There are the following layers : a normalization layer, 3 convolutional2D layers, 1 flatten layer and 4 fully-connected layers. ELU ([Exponential linear unit](https://www.quora.com/How-does-ELU-activation-function-help-convergence-and-whats-its-advantages-over-ReLU-or-sigmoid-or-tanh-function)) activation functions are added at each convolutional2D and fully-connected layers. The dropout regularization technique is inserted between the last convolutional2D layer and the flatten layer.   

The [final model](https://github.com/davisjazz/SDC_PRJ03_Cloning/blob/develop/_00_BSF/_Coding_/_1_pre-train/BSF_model_171203-2024.py) consisted of the following layers:   

| Layer         		  |     Description	        				         |   
|:-----------------------:|:------------------------------------------------:|    
| input         		  | 32 x 155 x 3 							         |      
| normalization           | lambda x: x/255.0-0.5                            |   
| convolution2D        	  | { filter: 16 ; stride: 2x2 ; kernel size : 5x5 } |   
| activation function	  | Exponential linear unit (ELU)                    |   
| convolution2D        	  | { filter: 32 ; stride: 2x2 ; kernel size : 5x5 } |   
| activation function	  | ELU                     			             |   
| convolution2D        	  | { filter: 64 ; stride: 2x2 ; kernel size : 5x5 } |   
| activation function	  | ELU                     			             |   
| regularization 		  | dropout(keep_prob = 0.5)  			             |   
| flatten	      		  |    			    			        	         |   
| fully connected (Dense) | Dense { 100 hidden units }   			         |   
| activation function	  | ELU                     			             |   
| fully connected (Dense) | Dense { 50 hidden units }   			         |   
| activation function	  | ELU                     			             |   
| fully connected (Dense) | Dense { 10 hidden units }   			         |   
| activation function	  | ELU                     			             |   
| fully connected (Dense) | Dense { 1 hidden unit }   			             |   


**Note:** I do not use a max pooling layer. For more details, read the [Geoffrey Hinton's comments on max pooling](https://mirror2image.wordpress.com/2014/11/11/geoffrey-hinton-on-max-pooling-reddit-ama/)

#### 2. Attempts to reduce overfitting in the model

The model contains one dropout layer in order to reduce overfitting.

The model was trained and validated on different data sets to ensure that the model was not overfitting (code line 44-50). The model was tested by running it through the simulator and ensuring that the vehicle could stay on the track.

#### 3. Model parameter tuning

The model used an adam optimizer, so the learning rate was not tuned manually (model.py line 70) except for the transfer learning phase where I changed the default learning rate from 1e-3 to 8.5e-4.   

#### 4. Appropriate training data

I tried to collect data using the keyboard arrows but it was very challenging and at the end the data were not as good as expected to train properly the neural network. Then the sample data provided by Udacity have been used to train the model. In order to keep the vehicle driving on the road they have been also preprocessed and jittered before fine tunning the neural network. Jitteration and preprocessing techniques have shown to be very effective ways to improve the neural network performances as I have learnt in the Traffic Sign Recognition project.  

For details about how I created the training data, see the next section. 

### Model Architecture and Training Strategy

#### 1. Solution Design Approach

It was an iterative approach with much trial and error.

Initially I used a convolution neural network model similar to the [Nvidia CNN](https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf) as discribed below:
![fig.2](https://raw.githubusercontent.com/davisjazz/SDC_PRJ03_Cloning/develop/_01_WIP/_Writeup_/_model_SPIN.PNG)


The Nvidia model might be appropriate because it learns to drive in track with or without lane markings or in areas with unclear visual guidance such as on unpaved roads. It automatically learns to detecte useful road features with only the steering angle as the training signal.


In order to gauge how well the model was working, I split my image and steering angle data into a training and validation set. I found that my first model had a low mean squared error on the training set but a high mean squared error on the validation set. This implied that the model was overfitting. 

To combat the overfitting, I used the following techniques:
- I trained the model with more data (augmentation and jitteration)
- I removed all images and the related steering angles driving the car off the track from the sample dataset
- I stopped the training when the loss has stopped improving
- I added a dropout layer as a regularization technique

If I would have more time, I would implement [the cross-validation technique](https://github.com/keras-team/keras/issues/1711) as well. But [the vehicle could stay on the track one](https://youtu.be/uxNCMQ3gk-Q) when the model was tested by running it through the simulator.

There were a few spots where the vehicle drove on the lane line of the track. To improve the driving behavior in these cases, I first tried to modify the convolutional neural network by removing a few convolutional2D and then I tweaked it.   

At first sight, [the car behavior was dramatically improved](https://youtu.be/_FEcHhhy9lc) by driving perfectly well in the center of the track from the beginning, then over the brige, passing successfully the first turn with its dirty border but it drove off the track in the second turn.   

From that point, I have fine tuned the model with more jittered data and I modified the compensation rate of the steering angle.

At the end of the process, [the vehicle is able to drive autonomously around the track without leaving the road](https://youtu.be/McrmB3ZIA18).

### VOS ETES ICI

#### 3. Creation of the Training Set & Training Process

To capture good driving behavior, I first recorded two laps on track one using center lane driving. Here is an example image of center lane driving:

![alt text][image2]

I then recorded the vehicle recovering from the left side and right sides of the road back to center so that the vehicle would learn to .... These images show what a recovery looks like starting from ... :

![alt text][image3]
![alt text][image4]
![alt text][image5]

Then I repeated this process on track two in order to get more data points.

To augment the data sat, I also flipped images and angles thinking that this would ... For example, here is an image that has then been flipped:

![alt text][image6]
![alt text][image7]

Etc ....

After the collection process, I had X number of data points. I then preprocessed this data by ...


I finally randomly shuffled the data set and put Y% of the data into a validation set. 

I used this training data for training the model. The validation set helped determine if the model was over or under fitting. The ideal number of epochs was Z as evidenced by ... I used an adam optimizer so that manually training the learning rate wasn't necessary.

### VOS ETES ICI





#### 3. Creation of the Training Set & Training Process

To capture good driving behavior, I first recorded two laps on track one using center lane driving. Here is an example image of center lane driving:

![alt text][image2]

I then recorded the vehicle recovering from the left side and right sides of the road back to center so that the vehicle would learn to .... These images show what a recovery looks like starting from ... :

![alt text][image3]
![alt text][image4]
![alt text][image5]

Then I repeated this process on track two in order to get more data points.

To augment the data sat, I also flipped images and angles thinking that this would ... For example, here is an image that has then been flipped:

![alt text][image6]
![alt text][image7]

Etc ....

After the collection process, I had X number of data points. I then preprocessed this data by ...


I finally randomly shuffled the data set and put Y% of the data into a validation set. 

I used this training data for training the model. The validation set helped determine if the model was over or under fitting. The ideal number of epochs was Z as evidenced by ... I used an adam optimizer so that manually training the learning rate wasn't necessary.


---
# ANNEXE

My model is an adaptation of the [nvidia architecture](https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf). 
>![fig.1](https://raw.githubusercontent.com/davisjazz/SDC_PRJ03_Cloning/develop/_01_WIP/_Writeup_/_model_1%20SPIN_2%20BSF.png)

My model consists of *a convolution neural network with 3x3 filter sizes and depths between 32 and 128 (model.py lines 18-24)*. It is an adaptation of the [nvidia architecture](https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf)

*The model includes ELU layers to introduce nonlinearity (code line 20), and the data is normalized in the model using a Keras lambda layer (code line 18)*.